## NDT-P2D

In [200]:
from sympy import symbols, Matrix, rot_axis1, rot_axis2, rot_axis3, hessian

x1, x2, x3 = symbols('x_1, x_2, x_3')
tx, ty, tz = symbols('t_x, t_y, t_z')
θx, θy, θz = symbols('θ_x, θ_y, θ_z')
# sympy provides rotation matrices of rotating the axes
# but we need to rotate the objects instead -> nagaive angles
Rx = rot_axis1(-θx)
Ry = rot_axis2(-θy)
Rz = rot_axis3(-θz)
x0 = Matrix([x1, x2, x3])
t = Matrix([tx, ty, tz])
p = Matrix([tx, ty, tz, θx, θy, θz])

x = Rx * Ry * Rz * x0 + t

dxdtx = x.jacobian(p).col(0)
dxdty = x.jacobian(p).col(1)
dxdtz = x.jacobian(p).col(2)
dxdθx = x.jacobian(p).col(3)  # 0, a, b in (6.18)
dxdθy = x.jacobian(p).col(4)  # c, d, e in (6.18)
dxdθz = x.jacobian(p).col(5)  # f, g, h in (6.18)

# Second partial derivative of dxdtx, dxdty, dxdtz are zeros
ddxdθxθx = dμdθx.jacobian(p).col(3)  # a in (6.20)
ddxdθxθy = dμdθx.jacobian(p).col(4)  # b in (6.20)
ddxdθxθz = dμdθx.jacobian(p).col(5)  # c in (6.20)
ddxdθyθy = dμdθy.jacobian(p).col(4)  # d in (6.20)
ddxdθyθz = dμdθz.jacobian(p).col(5)  # e in (6.20)
ddxdθzθz = dμdθz.jacobian(p).col(5)  # f in (6.20)

In [201]:
ddxdθxθx

Matrix([
[                                                                                                                                             0],
[ μ_{p1}*(-sin(θ_x)*sin(θ_y)*cos(θ_z) - sin(θ_z)*cos(θ_x)) + μ_{p2}*(sin(θ_x)*sin(θ_y)*sin(θ_z) - cos(θ_x)*cos(θ_z)) + μ_{p3}*sin(θ_x)*cos(θ_y)],
[μ_{p1}*(-sin(θ_x)*sin(θ_z) + sin(θ_y)*cos(θ_x)*cos(θ_z)) + μ_{p2}*(-sin(θ_x)*cos(θ_z) - sin(θ_y)*sin(θ_z)*cos(θ_x)) - μ_{p3}*cos(θ_x)*cos(θ_y)]])

## NDT-D2D

In [12]:
from sympy import symbols, Matrix, rot_axis1, rot_axis2, rot_axis3, hessian

μp1, μp2, μp3 = symbols('μ_{p1}, μ_{p2}, μ_{p3}')
tx, ty, tz = symbols('t_x, t_y, t_z')
θx, θy, θz = symbols('θ_x, θ_y, θ_z')
# sympy provides rotation matrices of rotating the axes
# but we need to rotate the objects instead -> nagaive angles
Rx = rot_axis1(-θx)
Ry = rot_axis2(-θy)
Rz = rot_axis3(-θz)
μp = Matrix([μp1, μp2, μp3])
t = Matrix([tx, ty, tz])
p = Matrix([tx, ty, tz, θx, θy, θz])

R = Rx * Ry * Rz
μ = R * μp + t  # -μq is a constant

# Derivative of Points
dμdtx = μ.jacobian(p).col(0)
dμdty = μ.jacobian(p).col(1)
dμdtz = μ.jacobian(p).col(2)
dμdθx = μ.jacobian(p).col(3)
dμdθy = μ.jacobian(p).col(4)
dμdθz = μ.jacobian(p).col(5)

# Second partial derivative of dμdtx, dμdty, dμdtz are zeros
ddμdθxθx = dμdθx.jacobian(p).col(3)
ddμdθxθy = dμdθx.jacobian(p).col(4)
ddμdθxθz = dμdθx.jacobian(p).col(5)
ddμdθyθy = dμdθy.jacobian(p).col(4)
ddμdθyθz = dμdθy.jacobian(p).col(5)
ddμdθzθz = dμdθz.jacobian(p).col(5)

# Derivative of Covariance, tx, ty, tz are zeros and then ignored
dRdθx = Matrix.zeros(3)
dRdθy = Matrix.zeros(3)
dRdθz = Matrix.zeros(3)
for i in range(3):
    for j in range(3):
        dRdθx[i, j] = R[i, j].diff(p)[3]
        dRdθy[i, j] = R[i, j].diff(p)[4]
        dRdθz[i, j] = R[i, j].diff(p)[5]

ddRdθxdθx = Matrix.zeros(3)
ddRdθxdθy = Matrix.zeros(3)
ddRdθxdθz = Matrix.zeros(3)
ddRdθydθy = Matrix.zeros(3)
ddRdθydθz = Matrix.zeros(3)
ddRdθzdθz = Matrix.zeros(3)
for i in range(3):
    for j in range(3):
        ddRdθxdθx[i, j] = dRdθx[i, j].diff(p)[3]
        ddRdθxdθy[i, j] = dRdθx[i, j].diff(p)[4]
        ddRdθxdθz[i, j] = dRdθx[i, j].diff(p)[5]
        ddRdθydθy[i, j] = dRdθy[i, j].diff(p)[4]
        ddRdθydθz[i, j] = dRdθy[i, j].diff(p)[5]
        ddRdθzdθz[i, j] = dRdθz[i, j].diff(p)[5]

# Small Angle Approximation by ORU
z = {θx: 0, θy: 0, θz: 0}

# Points
sdμdθx = dμdθx.subs(z)
sdμdθy = dμdθy.subs(z)
sdμdθz = dμdθz.subs(z)
sddμdθxθx = ddμdθxθx.subs(z)
sddμdθxθy = ddμdθxθy.subs(z)
sddμdθxθz = ddμdθxθz.subs(z)
sddμdθyθy = ddμdθyθy.subs(z)
sddμdθyθz = ddμdθyθz.subs(z)
sddμdθzθz = ddμdθzθz.subs(z)

# Covariances
sR = R.subs(z)
sdRdθx = dRdθx.subs(z)
sdRdθy = dRdθy.subs(z)
sdRdθz = dRdθz.subs(z)
sddRdθxdθx = ddRdθxdθx.subs(z)
sddRdθxdθy = ddRdθxdθy.subs(z)
sddRdθxdθz = ddRdθxdθz.subs(z)
sddRdθydθy = ddRdθydθy.subs(z)
sddRdθydθz = ddRdθydθz.subs(z)
sddRdθzdθz = ddRdθzdθz.subs(z)

a, b, c, d, e, f = symbols('a, b, c, d, e, f')
Σ = Matrix([[a, b, c], [b, d, e], [c, e, f]])
dΣdθx = sdRdθx * Σ * sR.T + sR * Σ * sdRdθx.T
dΣdθy = sdRdθy * Σ * sR.T + sR * Σ * sdRdθy.T
dΣdθz = sdRdθz * Σ * sR.T + sR * Σ * sdRdθz.T
ddΣdθxθx = sddRdθxdθx * Σ * sR.T + sR * Σ * sddRdθxdθx.T + sdRdθx * Σ * sdRdθx.T + sdRdθx * Σ * sdRdθx.T
ddΣdθxθy = sddRdθxdθy * Σ * sR.T + sR * Σ * sddRdθxdθy.T + sdRdθx * Σ * sdRdθy.T + sdRdθy * Σ * sdRdθx.T
ddΣdθxθz = sddRdθxdθz * Σ * sR.T + sR * Σ * sddRdθxdθz.T + sdRdθx * Σ * sdRdθz.T + sdRdθz * Σ * sdRdθx.T
ddΣdθyθy = sddRdθydθy * Σ * sR.T + sR * Σ * sddRdθydθy.T + sdRdθy * Σ * sdRdθy.T + sdRdθy * Σ * sdRdθy.T
ddΣdθyθz = sddRdθydθz * Σ * sR.T + sR * Σ * sddRdθydθz.T + sdRdθy * Σ * sdRdθz.T + sdRdθz * Σ * sdRdθy.T
ddΣdθzθz = sddRdθzdθz * Σ * sR.T + sR * Σ * sddRdθzdθz.T + sdRdθz * Σ * sdRdθz.T + sdRdθz * Σ * sdRdθz.T

In [16]:
vals = {μp1: 1, μp2: 4, μp3: 2, tx: 1, ty: 9, tz: 2, θx: 0.4, θy: 0.2, θz: 0.05, a: 0.05046745, b: -0.02039288, c: 0.02187753, d: 0.08852983, e: -0.06693526, f: 0.05210038}
print(dμdtx.subs(vals))
print(dμdty.subs(vals))
print(dμdtz.subs(vals))
print(dμdθx.subs(vals))
print(dμdθy.subs(vals))
print(dμdθz.subs(vals))
print('-----------')
print(ddμdθxθx.subs(vals))
print(ddμdθxθy.subs(vals))
print(ddμdθxθz.subs(vals))
print(ddμdθyθy.subs(vals))
print(ddμdθyθz.subs(vals))
print(ddμdθzθz.subs(vals))
print('-----------')
print((dRdθx * Σ * R.T + R * Σ * dRdθx.T).subs(vals))
print((dRdθy * Σ * R.T + R * Σ * dRdθy.T).subs(vals))
print((dRdθz * Σ * R.T + R * Σ * dRdθz.T).subs(vals))
print('-----------')
print((ddRdθxdθx * Σ * R.T + R * Σ * ddRdθxdθx.T + dRdθx * Σ * dRdθx.T + dRdθx * Σ * dRdθx.T).subs(vals))
print((ddRdθxdθy * Σ * R.T + R * Σ * ddRdθxdθy.T + dRdθx * Σ * dRdθy.T + dRdθy * Σ * dRdθx.T).subs(vals))
print((ddRdθxdθz * Σ * R.T + R * Σ * ddRdθxdθz.T + dRdθx * Σ * dRdθz.T + dRdθz * Σ * dRdθx.T).subs(vals))
print((ddRdθydθy * Σ * R.T + R * Σ * ddRdθydθy.T + dRdθy * Σ * dRdθy.T + dRdθy * Σ * dRdθy.T).subs(vals))
print((ddRdθydθz * Σ * R.T + R * Σ * ddRdθydθz.T + dRdθy * Σ * dRdθz.T + dRdθz * Σ * dRdθy.T).subs(vals))
print((ddRdθzdθz * Σ * R.T + R * Σ * ddRdθzdθz.T + dRdθz * Σ * dRdθz.T + dRdθz * Σ * dRdθz.T).subs(vals))

Matrix([[1], [0], [0]])
Matrix([[0], [1], [0]])
Matrix([[0], [0], [1]])
Matrix([[0], [-3.23441586268204], [3.02416383432189]])
Matrix([[1.80142942226922], [0.459610514794421], [-1.08708109407748]])
Matrix([[-3.96434991268384], [0.422832612756459], [1.05125750955378]])
-----------
Matrix([[0], [-3.02416383432189], [-3.23441586268204]])
Matrix([[0], [1.08708109407748], [0.459610514794421]])
Matrix([[0], [-1.05125750955378], [0.422832612756459]])
Matrix([[-1.18024875785162], [0.701509659406109], [-1.65922637430133]])
Matrix([[0.803613511568943], [-1.54379057132878], [3.65140807115183]])
Matrix([[-0.782910096261496], [-3.78747563851199], [-1.42901366993112]])
-----------
Matrix([[0, -0.00772869753412269, -0.0410639899547820], [-0.00772869753412269, 0.0520302433152153, 0.116856416373341], [-0.0410639899547820, 0.116856416373341, -0.0520302433152153]])
Matrix([[0.0462193454597939, -0.0474790636275670, -0.0418768853839271], [-0.0474790636275670, -0.0319821417935405, 0.0408321359874190], [-0.0

In [11]:
import numpy as np

np.cov(np.random.rand(3, 4))

array([[ 0.05046745, -0.02039288,  0.02187753],
       [-0.02039288,  0.08852983, -0.06693526],
       [ 0.02187753, -0.06693526,  0.05210038]])

## C++ Representations

In [304]:
from sympy import collect, expand
def Replace(a, coes):
    ret = ''
    for c in coes:
        ret += str(expand(a).coeff(c))\
            .replace('sin(θ_x)', 'sx').replace('cos(θ_x)', 'cx').replace('sin(θ_y)', 'sy')\
            .replace('cos(θ_y)', 'cy').replace('sin(θ_z)', 'sz').replace('cos(θ_z)', 'cz') + ', '
    return ret[:-2]

# Derivative of Points
μps = [μp1, μp2, μp3]
print(Replace(dμdθx[0], μps))
print(Replace(dμdθx[1], μps))
print(Replace(dμdθx[2], μps))

print(Replace(dμdθy[0], μps))
print(Replace(dμdθy[1], μps))
print(Replace(dμdθy[2], μps))

print(Replace(dμdθz[0], μps))
print(Replace(dμdθz[1], μps))
print(Replace(dμdθz[2], μps))

# Second partial derivative of dμdtx, dμdty, dμdtz are zeros
print('-------------')
print(Replace(ddμdθxθx[0], μps))
print(Replace(ddμdθxθx[1], μps))
print(Replace(ddμdθxθx[2], μps))

print(Replace(ddμdθxθy[0], μps))
print(Replace(ddμdθxθy[1], μps))
print(Replace(ddμdθxθy[2], μps))

print(Replace(ddμdθxθz[0], μps))
print(Replace(ddμdθxθz[1], μps))
print(Replace(ddμdθxθz[2], μps))

print(Replace(ddμdθyθy[0], μps))
print(Replace(ddμdθyθy[1], μps))
print(Replace(ddμdθyθy[2], μps))

print(Replace(ddμdθyθz[0], μps))
print(Replace(ddμdθyθz[1], μps))
print(Replace(ddμdθyθz[2], μps))

print(Replace(ddμdθzθz[0], μps))
print(Replace(ddμdθzθz[1], μps))
print(Replace(ddμdθzθz[2], μps))

0, 0, 0
-sx*sz + sy*cx*cz, -sx*cz - sy*sz*cx, -cx*cy
sx*sy*cz + sz*cx, -sx*sy*sz + cx*cz, -sx*cy
-sy*cz, sy*sz, cy
sx*cy*cz, -sx*sz*cy, sx*sy
-cx*cy*cz, sz*cx*cy, -sy*cx
-sz*cy, -cy*cz, 0
-sx*sy*sz + cx*cz, -sx*sy*cz - sz*cx, 0
sx*cz + sy*sz*cx, -sx*sz + sy*cx*cz, 0
-------------
0, 0, 0
-sx*sy*cz - sz*cx, sx*sy*sz - cx*cz, sx*cy
-sx*sz + sy*cx*cz, -sx*cz - sy*sz*cx, -cx*cy
0, 0, 0
cx*cy*cz, -sz*cx*cy, sy*cx
sx*cy*cz, -sx*sz*cy, sx*sy
0, 0, 0
-sx*cz - sy*sz*cx, sx*sz - sy*cx*cz, 0
-sx*sy*sz + cx*cz, -sx*sy*cz - sz*cx, 0
-cy*cz, sz*cy, -sy
-sx*sy*cz, sx*sy*sz, sx*cy
sy*cx*cz, -sy*sz*cx, -cx*cy
sy*sz, sy*cz, 0
-sx*sz*cy, -sx*cy*cz, 0
sz*cx*cy, cx*cy*cz, 0
-cy*cz, sz*cy, 0
-sx*sy*cz - sz*cx, sx*sy*sz - cx*cz, 0
-sx*sz + sy*cx*cz, -sx*cz - sy*sz*cx, 0


In [309]:
def Replace2(a):
    ret = ''
    for i in range(3):
        ret += str(expand(a[i]))\
            .replace('sin(θ_x)', 'sx').replace('cos(θ_x)', 'cx').replace('sin(θ_y)', 'sy')\
            .replace('cos(θ_y)', 'cy').replace('sin(θ_z)', 'sz').replace('cos(θ_z)', 'cz') + ', '
    return ret[:-2]

print(Replace2(dRdθx.row(0)))
print(Replace2(dRdθx.row(1)))
print(Replace2(dRdθx.row(2)))

print(Replace2(dRdθy.row(0)))
print(Replace2(dRdθy.row(1)))
print(Replace2(dRdθy.row(2)))

print(Replace2(dRdθz.row(0)))
print(Replace2(dRdθz.row(1)))
print(Replace2(dRdθz.row(2)))

print('-------------')
print(Replace2(ddRdθxdθx.row(0)))
print(Replace2(ddRdθxdθx.row(1)))
print(Replace2(ddRdθxdθx.row(2)))

print(Replace2(ddRdθxdθy.row(0)))
print(Replace2(ddRdθxdθy.row(1)))
print(Replace2(ddRdθxdθy.row(2)))

print(Replace2(ddRdθxdθz.row(0)))
print(Replace2(ddRdθxdθz.row(1)))
print(Replace2(ddRdθxdθz.row(2)))

print(Replace2(ddRdθydθy.row(0)))
print(Replace2(ddRdθydθy.row(1)))
print(Replace2(ddRdθydθy.row(2)))

print(Replace2(ddRdθydθz.row(0)))
print(Replace2(ddRdθydθz.row(1)))
print(Replace2(ddRdθydθz.row(2)))

print(Replace2(ddRdθzdθz.row(0)))
print(Replace2(ddRdθzdθz.row(1)))
print(Replace2(ddRdθzdθz.row(2)))

print('-------------')
print(Replace2(R.row(0)))
print(Replace2(R.row(1)))
print(Replace2(R.row(2)))

0, 0, 0
-sx*sz + sy*cx*cz, -sx*cz - sy*sz*cx, -cx*cy
sx*sy*cz + sz*cx, -sx*sy*sz + cx*cz, -sx*cy
-sy*cz, sy*sz, cy
sx*cy*cz, -sx*sz*cy, sx*sy
-cx*cy*cz, sz*cx*cy, -sy*cx
-sz*cy, -cy*cz, 0
-sx*sy*sz + cx*cz, -sx*sy*cz - sz*cx, 0
sx*cz + sy*sz*cx, -sx*sz + sy*cx*cz, 0
-------------
0, 0, 0
-sx*sy*cz - sz*cx, sx*sy*sz - cx*cz, sx*cy
-sx*sz + sy*cx*cz, -sx*cz - sy*sz*cx, -cx*cy
0, 0, 0
cx*cy*cz, -sz*cx*cy, sy*cx
sx*cy*cz, -sx*sz*cy, sx*sy
0, 0, 0
-sx*cz - sy*sz*cx, sx*sz - sy*cx*cz, 0
-sx*sy*sz + cx*cz, -sx*sy*cz - sz*cx, 0
-cy*cz, sz*cy, -sy
-sx*sy*cz, sx*sy*sz, sx*cy
sy*cx*cz, -sy*sz*cx, -cx*cy
sy*sz, sy*cz, 0
-sx*sz*cy, -sx*cy*cz, 0
sz*cx*cy, cx*cy*cz, 0
-cy*cz, sz*cy, 0
-sx*sy*cz - sz*cx, sx*sy*sz - cx*cz, 0
-sx*sz + sy*cx*cz, -sx*cz - sy*sz*cx, 0
